# Setup Private Link connection to Amazon Bedrock Service on EC2

This notebook provides steps to setup Private link connection to Bedrock on EC2. This notebook does not contain any interactive executable code. 

## Setup of VPC, Subnet, IAM Role

* Create an IAM Role for EC2 service with Bedrock and S3 access. For testing purpose, give full access to S3/ Bedrock

```
AmazonS3FullAccesss
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:*",
                "s3-object-lambda:*"
            ],
            "Resource": "*"
        }
    ]
}

bedrock_access_policy

{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Bedrock",
            "Effect": "Allow",
            "Action": "bedrock:*",
            "Resource": "*"
        }
    ]
}
```



Identify a VPC (or create one) where we will test the Private link. We need atleast one subnet under this VPC. Ensure the subnet has Route table associated with routes to internet. (NOTE: Only during the initial period to download pip packages and we will remove the route later)

<img src ="images/private_vpc_edit_routes.jpg" width="600"/>



* Create an EC2 Instance and launch it in this VPC/ Subnet with the IAM role created above
* Since this will be in a Private subnet (as we will remove the IGW association later), ensure the EC2 has Instance Connect Endpoint created.  You can create one from VPC → Endpoints → Create Endpoint → EC2 Instance Connect Endpoint


<img src ="images/private_vpc_create_vpc_endpoint.jpg" width="600"/>

* Create an interface endpoint for S3 from VPC → Endpoints → Create Endpoint → AWS Services. Filter for S3 and create one for S3 (interface endpoint).  Select the VPC and Subnet and enable DNS name

<img src ="images/private_vpc_interface_endpoint.jpg" width="600"/>

Once created, navigate to the endpoint details and copy the DNS name. We will use this later

<img src ="images/private_vpc_endpoint_detail.png" width="600"/>

## Download & Install Bedrock SDK



* Connect to EC2 instance and in the terminal, created a directory to download bedrock SDK

```
mkdir bedrock
cd bedrock
```

* Ensure you have python3-pip installed and perform an update

```
sudo yum update
sudo yum install python3-pip -y

if you have Ubuntu flavors, you will do sudo apt-get install python3-pip -y
```

* Create a python virtual environment

```
virtualenv bedrock_test
source bedrock_test/bin/activate
```

* Run the following command to download Bedrock SDK. This will download the Bedrock documentation, SDK in a folder

```
aws s3 cp s3://amazon-bedrock-limited-preview-documents/Documentation/ ./bedrock_docs --recursive --endpoint-url https://bucket.<paste DNS name from S3 interface endpoint here>
```

NOTE: If you get an error, check if EC2 is launched with proper IAM Role, IAM role has permissions to S3, S3 Interface endpoint DNS name is correctly mentioned in the command. 



* Run the following command to unzip the SDK to extract Python pip wheel files

```
unzip -o bedrock_docs/SDK/bedrock-python-sdk.zip -d bedrock_docs/SDK
```

* Run the following commands to install the wheel files in the virtual environment

```
python3 -m pip install bedrock_docs/boto3-1.26.162-py3-none-any.whl
python3 -m pip install bedrock_docs/botocore-1.29.162-py3-none-any.whl
```

* Once done, run the following command to quickly validate bedrock API. This will print the foundation models

```
aws bedrock list-foundation-models --region us-east-1 --endpoint-url https://bedrock.us-east-1.amazonaws.com
```

## Setup Bedrock Private-Link


Now, we will validate this with Private Link. To proceed further, create a VPC Endpoint for bedrock. Follow similar steps you did for S3 but filter for bedrock service in VPC → Endpoints → Create Endpoint → AWS Services. Ensure DNS name is enabled under Additional settings. 

<img src ="images/private_vpc_create_bedrock_endpoint.jpg" width="600"/>


In VPC console, remove the route to internet from the route table attached to subnet. 


<img src ="images/private_vpc_remove_routes.jpg" width="600"/>


Click save. After few mins, try running following command and it will not respond. You can cancel the command. 

```
curl https://www.amazon.com
```

Run the following command to quickly validate bedrock API. This should print the foundation models. 

```
aws bedrock list-foundation-models --region us-east-1 --endpoint-url https://bedrock.us-east-1.amazonaws.com
```

## Adding Endpoint Policy

We can add endpoint policies to control who can access the service via VPC endpoint. To perform this, navigate to endpoint details of Bedrock interface endpoint and add the following condition to the policy. Click policy tab and Edit Policy and add the custom policy


```
{
    "Statement": [
        {
            "Action": "*",
            "Effect": "Allow",
            "Principal": "*",
            "Resource": "*",
            "Condition": {
                "ArnEquals": {
                    "aws:PrincipalArn": "<ARN of EC2 IAM Role>"
                }
            }
        }
    ]
}
```

To validate the policy working, create another IAM role with permissions to Bedrock and switch EC2 instance to the second role from EC2 console. Actions → Security → Modify IAM Role. Wait for few minutes and when you run the following command you will get an error


```
aws bedrock list-foundation-models --region us-east-1 --endpoint-url https://bedrock.us-east-1.amazonaws.com
```

<img src ="images/private_vpc_test_endpoint.jpg" width="600"/>


You can alternatively create a Python file and run the program


```
import boto3 
bedrock = boto3.client('bedrock' , 'us-east-1', endpoint_url='https://bedrock.us-east-1.amazonaws.com')')
print(bedrock.list_foundation_models())
```

```
python3 test_endpoint.py
```

<img src ="images/private_vpc_list_fm.jpg" width="600"/>





## Troubleshooting Tips

If you get unknown service error or aws: error: argument command: Invalid choice, valid choices are,  check boto3 install location & validate if bedrock package is installed properly


```
pip show boto3
```

<img src ="images/private_vpc_show_boto3.jpg" width="600"/>

Execute the following command and check if Bedrock is available

```
ls /home/ec2-user/bedrock/bedrock_test/lib/python3.9/site-packages/botocore/data | grep bedrock
```